In [50]:
import torch.cuda

from src import  N_CLASSES, FEATURE_DIM

BATCH_SIZE = 100
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'device: {device}')

device: cpu


In [51]:
# 读数据
data = torch.load("data.pth")
feature = data['feature']
label = data['label']

In [52]:
# 生成训练集、测试集
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.2, train_size=0.8, random_state=0)
x_train, x_test, y_train, y_test = x_train.to(device), x_test.to(device), y_train.to(device), y_test.to(device)
# x_train = feature.to(device)
# y_train = label.to(device)

In [53]:
# 加载数据
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [54]:
# 定义网络
from model import Net
import torch.nn as nn

net = Net().to(device)

# 定义损失函数、优化器
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),
                       lr=0.001,
                       betas=(0.9, 0.999),
                       eps=1e-08,
                       weight_decay=0,
                       amsgrad=False)

In [55]:
# 训练网络
for epoch in range(500):  # 数字越大，循环次数越多
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    if epoch % 10 == 0:
        print(f'epoch {epoch} loss: {running_loss / 1500:.10f}')
        running_loss = 0.0

print('Finished Training')

# 训练结束后保存网络参数
PATH = './model.pth'
torch.save(net.state_dict(), PATH)

epoch 0 loss: 0.1022655870
epoch 10 loss: 0.0091516626
epoch 20 loss: 0.0051610491
epoch 30 loss: 0.0032670550
epoch 40 loss: 0.0023635755
epoch 50 loss: 0.0024437057
epoch 60 loss: 0.0019017887
epoch 70 loss: 0.0014202280
epoch 80 loss: 0.0013623403
epoch 90 loss: 0.0013394938
epoch 100 loss: 0.0011808470
epoch 110 loss: 0.0012995278
epoch 120 loss: 0.0012470213
epoch 130 loss: 0.0008711648
epoch 140 loss: 0.0008137441
Finished Training


In [56]:
# 测试模型
net = Net().to(device)
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    # noinspection PyShadowingBuiltins
    for input, labels in test_loader:
        # calculate outputs by running images through the network
        outputs = net(input)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # noinspection PyUnresolvedReferences
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total}')

Accuracy: 0.9744166666666667
